In [1]:
from random import *


class Game: #A class for managing different situations and states happening in the game and on the board
    def __init__(self, Mat, dimX, dimY):
        self.Mat = Mat
        self.dimX = dimX
        self.dimY = dimY

    def Initiate(self): #initiating the game board with X and Y dimensions
        for i in range(0, self.dimY):
            R = []
            for j in range (0, self.dimX):
                if i % 2 == 1 and j % 2 == 1:
                    R.append(randint(1, 9))  # Assigning a random number from 1 to 9 to the spots in the board as the points
                elif i % 2 == 0 and j % 2 == 0:
                    R.append('*') # printing asterisks as the dots in the board
                else:
                    R.append(' ') # adding extra space for actions in the game
            self.Mat.append(R)

    def Get_matrix(self): # Board matrix
        ans = []
        for i in range(0, self.dimY):
            R = []
            for j in range(0, self.dimX):
                R.append(self.Mat[i][j])
            ans.append(R)
        return ans

    def Draw_mat(self): # Drawing the board marix as dots and lines
        
        if self.dimX > 9:
            print(" ", end='')
        print("   ", end='')
        for i in range(0, self.dimX):
            print(str(i), end='  ')
        print()

        if self.dimX > 9:
            print(" ", end='')
        print("   ", end='')
        for i in range(0, self.dimX + 1):
            print("___", end='')
        print()
        for j in range(self.dimY):
            if self.dimX > 9 and j < 10:
                print(" ", end='')
            print(str(j) + "| ", end='')
            for z in range(self.dimX):
                print(str(self.Mat[j][z]), end='  ')
            print()
        print("   ________________________\n")

    def Get_currentState(self):
        return Game(self.Get_matrix(), self.dimX, self.dimY)

    def action(self, i, j): # Applying the actions made by the human or the computer
        Sum = 0

        if j % 2 == 0 and i % 2 == 1:
            self.Mat[j][i] = '-'
            if j < self.dimY - 1:
                if self.Mat[j+2][i] == '-' and self.Mat[j+1][i+1] == '|' and self.Mat[j+1][i-1] == '|':
                    Sum += self.Mat[j+1][i]
            if j > 0:
                if self.Mat[j-2][i] == '-' and self.Mat[j-1][i+1] == '|' and self.Mat[j-1][i-1] == '|':
                    Sum += self.Mat[j-1][i]

        else:
            self.Mat[j][i] = '|'
            if i < self.dimX - 1:
                if self.Mat[j][i+2] == '|' and self.Mat[j+1][i+1] == '-' and self.Mat[j-1][i+1] == '-':
                    Sum += self.Mat[j][i+1]
            if i > 0:
                if self.Mat[j][i-2] == '|' and self.Mat[j+1][i-1] == '-' and self.Mat[j-1][i-1] == '-':
                    Sum += self.Mat[j][i-1]
        return Sum


In [2]:

class Algo: # A class for defining algorithms used (minimax and alpha-beta pruning)
    
    def miniMax(State, depth): # Function for the minimax algorithm

        for i in range(State.Current.dimY):
            for j in range(State.Current.dimX):
                if State.Current.Mat[i][j] == ' ' and (j, i) not in State.children:
                    State.Make(j, i, True)
                    if depth < 2:
                        return (i, j)

        Minimum_Score = 1000
        i = 0
        j = 0
        for k, z in State.children.items():
            Result = Algo.Maximum(z, depth - 1, Minimum_Score)
            if Minimum_Score > Result:
                Minimum_Score = Result
                i = k[0]
                j = k[1]

        return (i, j)


    def Maximum(State, depth, Alpha): # Alpha-beta pruning function for taking care of Alpha values
        if depth == 0:
            return State.CurrentScore

        for i in range(State.Current.dimY):
            for j in range(State.Current.dimX):
                if State.Current.Mat[i][j] == ' ' and (j, i) not in State.children:
                    State.Make(j, i, False)

        Maximum_Score = -1000
        i = 0
        j = 0
        for k, z in State.children.items():
            Result = Algo.Minimum(z, depth - 1, Maximum_Score)
            if Maximum_Score < Result:
                Maximum_Score = Result
            if Result > Alpha:
                return Result

        return Maximum_Score


    def Minimum(State, depth, Beta): # Alpha-beta pruning function for taking care of Beta values
        if depth == 0:
            return State.CurrentScore

        for i in range(State.Current.dimY):
            for j in range(State.Current.dimX):
                if State.Current.Mat[i][j] == ' ' and (j, i) not in State.children:
                    State.Make(j, i, True)

        Minimum_Score = 1000
        i = 0
        j = 0
        for k, z in State.children.items():
            Result = Algo.Maximum(z, depth - 1, Minimum_Score)
            if Minimum_Score > Result:
                Minimum_Score = Result
            if Result < Beta:
                return Result

        return Minimum_Score


In [4]:
class Thing: # A class for Node related operations
    def __init__(self, currentState):
        self.Current = currentState
        self.CurrentScore = 0
        self.children = {}

    def Make(self, i, j, player): # Function for generating a child node
        self.children[(i, j)] = Thing(self.Current.Get_currentState())
        mul = 1
        if player:
            mul *= -1
        self.children[(i, j)].CurrentScore = (self.children[(i, j)].Current.action(i, j) * mul) + self.CurrentScore

    def Populate(self, i, j, Child): # Function for adding a node
        self.children[(i,j)] = Child

    def Draw(self): # function for drawing the board
        self.Current.Draw_mat()


In [3]:
from flask import Flask, request, jsonify
from random import *
import collections
from Algorithm import *
from Board import *
from Nodes import *

app = Flask(__name__)

class DotsNBoxes: 
    def __init__(self, Board_Xdim, Board_Ydim, Ply_num):
        currentState = Game([], Board_Xdim, Board_Ydim)
        currentState.Initiate()
        self.State = Thing(currentState)
        self.Ply_num = Ply_num
        self.Score = 0

    def start_game(self, board_rows, board_cols, ply_num):
        self.State.Draw()
        self.Ply_num = ply_num
        return {"message": "Game started!"}

    def make_move(self, human_x, human_y):
        if (human_x, human_y) not in self.State.children:
            self.State.Make(human_x, human_y, False)
            self.State = self.State.children[(human_x, human_y)]
        else:
            self.State = self.State.children[(human_x, human_y)]
        result = self.computer_move()
        self.State.Draw()
        return result

    def computer_move(self):
        move = Algo.miniMax(self.State, self.Ply_num)
        self.State = self.State.children[(move[0], move[1])]
        return {
            "computer_move": {
                "x": move[0],
                "y": move[1]
            },
            "current_score": self.State.CurrentScore
        }


game=None
@app.route("/start", methods=["POST"])
def start_game():
    global game
    data = request.get_json()
    board_rows = data.get("board_rows")
    board_cols = data.get("board_cols")
    ply_num = data.get("ply_num")
    game = DotsNBoxes(board_rows * 2 + 1, board_cols * 2 + 1, ply_num)
    return jsonify(game.start_game(board_rows * 2 + 1, board_cols * 2 + 1, ply_num))

@app.route("/move", methods=["POST"])
def make_move():
    global game
    data = request.get_json()
    human_x = data.get("x")
    human_y = data.get("y")

    return jsonify(game.make_move(human_x, human_y))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=4000,)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4000
 * Running on http://192.168.1.7:4000
Press CTRL+C to quit
192.168.1.7 - - [18/Mar/2024 20:00:16] "POST /start HTTP/1.1" 200 -


   0  1  2  3  4  5  6  
   ________________________
0| *     *     *     *  
1|    9     2     4     
2| *     *     *     *  
3|    8     3     1     
4| *     *     *     *  
5|    2     1     7     
6| *     *     *     *  
   _________________________



192.168.1.7 - - [18/Mar/2024 20:00:17] "POST /move HTTP/1.1" 200 -


   0  1  2  3  4  5  6  
   ________________________
0| *  -  *     *     *  
1|    9     2     4     
2| *     *  -  *     *  
3|    8     3     1     
4| *     *     *     *  
5|    2     1     7     
6| *     *     *     *  
   _________________________



...

   0  1  2  3  4  5  6  
   ________________________
0| *     *     *     *  
1|    5     6     1     
2| *     *     *     *  
3|    9     1     3     
4| *     *     *     *  
5|    4     2     9     
6| *     *     *     *  
   _________________________



192.168.1.7 - - [18/Mar/2024 20:13:54] "POST /start HTTP/1.1" 200 -
192.168.1.7 - - [18/Mar/2024 20:13:54] "POST /start HTTP/1.1" 200 -


   0  1  2  3  4  5  6  
   ________________________
0| *     *     *     *  
1|    4     4     2     
2| *     *     *     *  
3|    5     3     3     
4| *     *     *     *  
5|    7     7     8     
6| *     *     *     *  
   _________________________

   0  1  2  3  4  5  6  
   ________________________
0| *     *     *     *  
1|    7     4     1     
2| *     *     *     *  
3|    9     5     4     
4| *     *     *     *  
5|    4     2     6     
6| *     *     *     *  
   _________________________



192.168.1.7 - - [18/Mar/2024 20:14:55] "POST /start HTTP/1.1" 200 -


   0  1  2  3  4  5  6  
   ________________________
0| *     *     *     *  
1|    4     1     6     
2| *     *     *     *  
3|    2     9     4     
4| *     *     *     *  
5|    5     9     3     
6| *     *     *     *  
   _________________________

